In [1]:
# import necessary packages
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv
from PIL import Image
import requests
from io import BytesIO
from difflib import SequenceMatcher
from skimage.io import imshow
import warnings
import random
import itertools

In [2]:
!git clone https://github.com/dwyl/english-words

Cloning into 'english-words'...
remote: Enumerating objects: 248, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 248 (delta 97), reused 83 (delta 83), pack-reused 123
Receiving objects: 100% (248/248), 27.63 MiB | 19.54 MiB/s, done.
Resolving deltas: 100% (117/117), done.


# Definitions

## Words

In [3]:
with open('/content/english-words/words_alpha.txt') as word_file:
  words = set(word_file.read().split())

consonants = 'bcdfghjklmnpqrstvwxyz'
vowels = 'aeiou'

In [4]:
def sub_word(word, letters):
  for letter in word:
    if letter not in letters or word.count(letter) > letters.count(letter):
      return False
  return True

def get_letters(vowel_consonant_pattern):
  output = ""
  for char in vowel_consonant_pattern:
    if char == 'v':
      output += random.choice(vowels)
    else:
      assert char == 'c'
      output += random.choice(consonants)
  print(f"Letters are {output}")
  return output

def get_best_word(letters):
  max_len = 0
  for word in words:
    if sub_word(word, letters):
      if len(word) > max_len:
        max_len = len(word)

  max_words = []

  for word in words:
    if sub_word(word, letters) and len(word) == max_len:
      max_words.append(word)

  return max_words

def letters_round(letters, above_letters=0):
  assert(len(letters) == 9)
  return_words = [[] for i in range(9)]

  for word in words:
    if sub_word(word, letters):
      return_words[len(word) - 1].append(word)

  for i, word_list in enumerate(return_words[::-1]):
    if 9 - i > above_letters:
      print(f"{9-i} letter words:")
      for word in word_list:
        print(word)
      if len(word_list) == 0:
        print("None found")

## Numbers

### This way isn't totally comprehensive but is quite fast

In [5]:
def symbol_combinations(num_symbols):

  symbols = ['+', '-', '*', '/']
  all_combinations = itertools.product(symbols, repeat=num_symbols)
  output = [''.join(combination) for combination in all_combinations]

  return output

def possible_orders(num_list):
  outcomes = []

  for r in range(1, len(num_list) + 1):
    for permutation in itertools.permutations(num_list, r):
      this_symbols = symbol_combinations(r - 1)
      for symbols in this_symbols:
        this_outcome = ""
        for i in range(len(permutation) - 1):
          this_outcome += str(permutation[i]) + symbols[i]
        this_outcome += str(permutation[-1])

        outcomes.append(this_outcome)

  evaluations = [eval(outcome) for outcome in outcomes]
  return np.array([outcomes, evaluations])

def numbers_round(numbers, target):
  results = possible_orders(numbers)
  answers = np.where(results[1] == str(target))
  if len(answers[0]) == 0:
    print("No result found")
  else:
    print(f"{len(answers[0])} results found")
    for answer in results[0][answers]:
      print(answer)

### This way gets more values but is slower

In [6]:
def possible_numbers(num_list):
  if len(num_list) < 2:
    return num_list

  if len(num_list) == 2:
    a = num_list[0]
    b = num_list[1]
    outcomes = [a, b, a + b, a - b, a * b]
    if (not b == 0) and int(a / b) == a / b:
      outcomes.append(int(a / b))

    if (not a == 0) and int(b / a) == b / a:
      outcomes.append(int(b / a))

  else:
    outcomes = []

    for r in range(1, len(num_list) + 1):
      for permutation in itertools.permutations(num_list, r):
        for i in range(len(permutation) - 1):
          this_outcomes = possible_numbers([permutation[i], permutation[i+1]])
          for each_outcome in this_outcomes:
            outcomes += possible_numbers(list(permutation[0:i]) + [each_outcome] + list(permutation[i+2:]))

  return list(set(outcomes))

def useful_numbers(num_list):
  all_numbers = possible_numbers(num_list)
  useful_numbers = []
  for num in all_numbers:
    if num > 100 and num < 1000:
      useful_numbers.append(num)

  return useful_numbers

# Tests

In [7]:
letters_round(get_letters('ccvcvcvvc'), above_letters=5)

Letters are bsesemuel
9 letter words:
None found
8 letter words:
None found
7 letter words:
meubles
bemuses
beseems
6 letter words:
smeuse
lessee
sebums
emeses
beseem
bemuse
blesse
semese
melees
umbles
semble
bessel
umbels
blumes
mussel


In [8]:
letters_round('petrovild', above_letters=5)

9 letter words:
None found
8 letter words:
None found
7 letter words:
poitrel
dropvie
prevoid
provide
viroled
pretold
protide
diopter
pivoted
leproid
pteroid
pivoter
dioptre
droplet
leporid
tripled
overtip
piloted
proetid
politer
peridot
proteid
overlip
6 letter words:
violer
divert
dipole
toivel
toiler
porite
piolet
divort
petrol
redipt
livedo
ported
replod
virled
triole
leprid
rioted
podite
polder
dopier
leptid
dolite
diploe
loiter
pioted
roiled
dotier
devoir
triple
virole
peloid
protei
perdit
violet
potlid
rodlet
retold
polite
oliver
drivel
plover
torpid
dipter
editor
ortive
revolt
idolet
pretil
privet
edplot
tirled
triode
tripel
epirot
period
deport
proved
redtop
prevot
tripod
lovier
olived
podler
replot
protid
voider
trepid
optive
toiled
liepot
torvid
olivet
perlid


In [12]:
numbers_round([75, 50, 5, 6, 10], 309)

36 results found
75+50*5-6-10
75+50*5-10-6
75+5*50-6-10
75*5-50-6-10
75+5*50-10-6
75*5-50-10-6
75*5-6-50-10
75*5-6-10-50
75*5-10-50-6
75*5-10-6-50
75-6+50*5-10
75-6+5*50-10
75-6-10+50*5
75-6-10+5*50
75-10+50*5-6
75-10+5*50-6
75-10-6+50*5
75-10-6+5*50
50*5+75-6-10
50*5+75-10-6
50*5-6+75-10
50*5-6-10+75
50*5-10+75-6
50*5-10-6+75
5*75-50-6-10
5*75-50-10-6
5*75-6-50-10
5*75-6-10-50
5*75-10-50-6
5*75-10-6-50
5*50+75-6-10
5*50+75-10-6
5*50-6+75-10
5*50-6-10+75
5*50-10+75-6
5*50-10-6+75
